# API from skyscanner
The objective of this part is to extract data from Skyscanner about the flights from
    A. Berlin TXL to Cancun CUN
    B. Berlin TXL to Rio de Janeiro GIG 
    C. Berlin TXL to Buenos Aires EZE
For 3 different dates:
    A. 21-12-2019 - 27-12-2019
    B. 18-01-2020 - 24-01-2020
    C. 22-02-2020 - 28-02-2020

In [7]:
# Importing modules
import pandas as pd
import numpy as np
import requests
import json
import matplotlib
%matplotlib inline

# Defining a dictionary of International Airports
airports = {"ATL":"Hartsfield",
            "PEK":"Beijing",
            "DXB":"Dubai",
            "LAX":"Los Angeles",
            "HND":"Tokyo",
            "ORD":"Chicago",
            "LHR":"London",
            "SAR":"Hong Kong International Airport",
            "PVG":"Shanghai",
            "CDG":"Paris",
            "AMS":"Amsterdam",
            "DEL":"Delhi",
            "CAN":"Guangzhou",
            "FRA":"Frankfur",
            "DFW":"Dallas",
            "ICN":"Seoul",
            "IST":"Istanbul",
            "CGK":"Soekarno-Hatta",
            "SIN":"Singapore",
            "DEN":"Denver",
            "CUN": "Cancun",
            "GIG": "Rio de Janeiro",
            "EZE": "Buenos Aires"
           }

""" Function that takes 3 arguments: departure date, return date and final destination.
 Based on the arguments it extracts the live data from Skyscanner.com for these inputs and
 returns a dataframe with the cheapest flight informations"""

def get_cheapest_flights(departure_date, return_date, final_destination):

# Requesting key pass via API to search live flight information

    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/v1.0"

    payload = ('inboundDate='
               +return_date
               +'&children=0&infants=0&country=US&currency=EUR&locale=en-US&originPlace=TXL-sky&destinationPlace='
               +final_destination
               +'-sky&outboundDate='
               +departure_date+'&adults=1')
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a",
    'content-type': "application/x-www-form-urlencoded"
    }

    response = requests.request("POST", url, data=payload, headers=headers)

    key_pass = '/'.join(dict(response.headers)['Location'].split('/')[3:])

# Requesting live flight data by API using the key pass

    base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/"
    url = base_url + key_pass
    querystring = {"pageIndex":"0","pageSize":"10", "sortType": "price"} # already sorting by price

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': "7eeab0197bmshf3507811dfa3ebap1e6084jsne17b9d61825a"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    
# Creating Pandas dataframes from retrieved response.json() : 
# iterinaries, flight detailed informations (leg arrival and departure)

    iterinaries = pd.DataFrame(response.json()['Itineraries'])
    flight_agents = pd.DataFrame(response.json()['Agents'])
    
    legs_departure = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration','Stops']]
                         .rename(columns={'Id': 'OutboundLegId',
                             'Departure': 'Outbound_departure',
                             'Arrival': 'Outbound_arrival',
                             'Duration': 'Outbound_duration',
                             'Stops': 'Outbound_stops'}))

    legs_arrival = (pd.DataFrame(response
                         .json()['Legs'])[['Id','Departure','Arrival','Duration', 'Stops']]
                         .rename(columns={'Id': 'InboundLegId',
                             'Departure': 'Inbound_departure',
                             'Arrival': 'Inbound_arrival',
                             'Duration': 'Inbound_duration',
                             'Stops': 'Inbound_stops'}))
    
# Merging iterinaties and flight information into one dataframe df
    
    df = (iterinaries
          .merge(legs_departure, on = 'OutboundLegId')
          .merge(legs_arrival, on = 'InboundLegId')
          .drop(columns=['OutboundLegId', 'InboundLegId', 'BookingDetailsLink'])
          )
# Extracting data about the ticket providers (Agents), flight price and purchase link
    
    df['Agents'] = [i[0]['Agents'][0] for i in df['PricingOptions']]
    df.insert(2, 'Flight_price', [i[0]['Price'] for i in df['PricingOptions']]) 
    df['Link']  = [i[0]['DeeplinkUrl'] for i in df['PricingOptions']]
    
# Extracting the number of stops for each flight

    df['Outbound_stops'] = df['Outbound_stops'].str.len()
    df['Inbound_stops'] = df['Inbound_stops'].str.len

# Converting outbound/inbound departure into two columns with the date and the time separately

    df.insert(3,'Outbound_departure_date', pd.to_datetime(df['Outbound_departure']).dt.date)
    df.insert(4,'Outbound_departure_time', pd.to_datetime(df['Outbound_departure']).dt.time)
    df.insert(8,'Inbound_departure_date',  pd.to_datetime(df['Inbound_departure']).dt.date)
    df.insert(9,'Inbound_departure_time',pd.to_datetime(df['Inbound_departure']).dt.time)

    df['Outbound_duration'] = pd.to_datetime(df['Outbound_duration'], unit='m').dt.strftime('%H:%M')
    df['Inbound_duration'] = pd.to_datetime(df['Inbound_duration'], unit='m').dt.strftime('%H:%M')
    
# Accessing the name of ticket prividers
              
    flight_agents = flight_agents[['Id', 'Name']]

    df = df.rename(columns={'Agents': 'Id'})
    df = df.merge(flight_agents, on = 'Id')
    df = df.rename(columns={'Name': 'Ticket_provider'})
              
# Final changes, removing unnecassary columns and inserting 2 new columns with the identifier and the destination          
              
    df = df.drop(columns=['PricingOptions','Outbound_departure' ,'Outbound_arrival','Inbound_departure'	,'Inbound_arrival', 'Id'])
    
    df.insert(1, 'Destination', airports[final_destination])
    df.insert(0,'Id', [airports[final_destination]+'_'+departure_date]*len(df))
    
    return(df.sort_values('Flight_price')[:1])

""" Function that asks a user to input the destination as weel as the dates they are 
considering to travel to"""              
              
def query_questions():
    
        final_destination = input('3 destinations you are considering? FORMAT: Cancun,Rio de Janeiro,Buenos Aires').split(',')
        departure_date = input('3 dates you want to depart? FORMAT: 2019-12-21,2020-01-18,2020-02-22').split(',')
        return_date = input('3 dates you want to come back? FORMAT 2019-12-27,2020-01-24,2020-02-28').split(',')
        return(final_destination,departure_date,return_date)
              
query_output = query_questions()

# Converting the destination places to aiport abbrevations, that the flight search feeds on
# And saving it into variable final destination
              
final_destination = query_output[0]
destinations = []
for i in airports.items():
    for j in final_destination:
        if j in i:
            destinations.append(i[0])
final_destination = destinations

# Saving the departure date and return date retrieved from the query questions into variables  
              
departure_date = query_output[1]
return_date = query_output[2] 

""" Function that uses user's destinaton, departure and return dates of choice """ 
              
def query_search(destinations, dep_date, return_date):
    b = pd.DataFrame()
    m = 0
    r = 0
    for i in range(len(dep_date)):
        n = 0
        for j in range(len(destinations)):
            a = [dep_date[n]]
            a.append(return_date[n])
            a.append(destinations[m])
            n += 1
            b = b.append(get_cheapest_flights( a[r], a[r+1], a[r+2]))
        m += 1
    return(b.reset_index(drop=True))

final_output = query_search(final_destination, departure_date , return_date)

# Saving the csv file and reading it

output_file = input('All done! How do you want to save your file? FORMAT: data_thieves_skyscanner_final_2.csv')
final_output.to_csv(output_file, index=False)
flights = pd.read_csv(output_file)
flights

3 destinations you are considering? FORMAT: Cancun,Rio de Janeiro,Buenos AiresCancun,Rio de Janeiro,Buenos Aires
3 dates you want to depart? FORMAT: 2019-12-21,2020-01-18,2020-02-222019-12-21,2020-01-18,2020-02-22
3 dates you want to come back? FORMAT 2019-12-27,2020-01-24,2020-02-282019-12-27,2020-01-24,2020-02-28
All done! How do you want to save your file? FORMAT: data_thieves_skyscanner_final_2.csvdata_thieves_skyscanner_final_3.csv


,Id,Flight_price,Destination,Outbound_departure_date,Outbound_departure_time,Outbound_duration,Outbound_stops,Inbound_departure_date,Inbound_departure_time,Inbound_duration,Inbound_stops,Link,Ticket_provider
0,Cancun_2019-12-21,1577.87,Cancun,2019-12-21,07:45:00,16:50,2,2019-12-27,06:45:00,19:25,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Expedia
1,Cancun_2020-01-18,700.41,Cancun,2020-01-18,09:00:00,17:10,2,2020-01-24,12:55:00,17:00,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Expedia
2,Cancun_2020-02-22,806.55,Cancun,2020-02-22,09:55:00,19:40,1,2020-02-28,15:13:00,17:52,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Kissandfly
3,Rio de Janeiro_2019-12-21,1604.20,Rio de Janeiro,2019-12-21,19:45:00,02:55,1,2019-12-27,20:40:00,18:25,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,FlightNetwork
4,Rio de Janeiro_2020-01-18,1011.08,Rio de Janeiro,2020-01-18,06:25:00,15:50,1,2020-01-24,16:30:00,14:50,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Air France
5,Rio de Janeiro_2020-02-22,947.93,Rio de Janeiro,2020-02-22,14:30:00,20:25,2,2020-02-28,16:30:00,17:15,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Vayama
6,Buenos Aires_2019-12-21,1742.19,Buenos Aires,2019-12-21,12:30:00,23:55,1,2019-12-27,14:35:00,17:10,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Iberia
7,Buenos Aires_2020-01-18,1263.37,Buenos Aires,2020-01-18,19:45:00,16:40,1,2020-01-24,23:50:00,17:30,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,Travel2Be
8,Buenos Aires_2020-02-22,998.57,Buenos Aires,2020-02-22,12:15:00,20:25,1,2020-02-28,22:45:00,08:40,<bound method _noarg_wrapper.<locals>.wrapper ...,http://partners.api.skyscanner.net/apiservices...,GotoGate
